## 1. Imports

In [69]:
import yaml
import pandas as pd
import numpy as np
import pmaw
import torch
from sklearn.metrics import cohen_kappa_score
from statistics import multimode, mode
from collections import Counter
from statsmodels.stats.inter_rater import fleiss_kappa

In [72]:
df = pd.read_excel("../../data/external/annotations_wsb.xlsx", sheet_name="annotations_scale")

In [73]:
df.dropna(inplace=True)

In [15]:
df

,id,ben,dliden,doss,jmo,jmo2,post,Own annotation
0,h15nzwn,1,1,1,1,1,"\n \nGentlemen, start your boners:\n\nBlackBe...",1.0
1,gyxowrt,1,1,1,1,1,#It has come to my attention that $PLTR Nation...,1.0
2,h0cufvd,1,1,1,1,1,#SNDL and TLRY for smooking weed on the moon 🚬...,1.0
3,h16r93i,1,1,1,1,1,$63k in on WKHS calls,1.0
4,h1adyxq,0,1,0,1,1,$CLF and $MT,0.0
...,...,...,...,...,...,...,...,...
178,gx0pu7v,1,1,1,1,1,Woke up on the floor got up and I looked perfe...,1.0
179,gvgbq2j,-1,-1,-1,1,-1,"Yeah, I was going to sell my UWMC for a L, but...",-1.0
180,h0g8s39,1,1,1,1,1,"Yep, it’s cheap right now and we could see it ...",1.0
181,h12l678,0,0,0,-1,-1,You didn’t know what a hedge fund was before G...,0.0


In [24]:
score = cohen_kappa_score(df["jmo2"], df["Own annotation"])

In [50]:
def majority_vote(annotator_results_list: list):
    return mode(annotator_results_list)


In [51]:
annotators = ["ben", "dliden", "doss", "jmo", "jmo2"]

In [54]:
df["majority vote"] = df[annotators].apply(lambda x: majority_vote(x), axis=1)

In [43]:
test_list = multimode([1,2,2,3,4,4,3])

In [44]:
test_list

[2, 3, 4]

In [39]:
data = Counter(test_list)

In [41]:
data.most_common(1)

[(2, 2)]

In [55]:
df

,id,ben,dliden,doss,jmo,jmo2,post,Own annotation,majority vote
0,h15nzwn,1,1,1,1,1,"\n \nGentlemen, start your boners:\n\nBlackBe...",1.0,1
1,gyxowrt,1,1,1,1,1,#It has come to my attention that $PLTR Nation...,1.0,1
2,h0cufvd,1,1,1,1,1,#SNDL and TLRY for smooking weed on the moon 🚬...,1.0,1
3,h16r93i,1,1,1,1,1,$63k in on WKHS calls,1.0,1
4,h1adyxq,0,1,0,1,1,$CLF and $MT,0.0,1
...,...,...,...,...,...,...,...,...,...
178,gx0pu7v,1,1,1,1,1,Woke up on the floor got up and I looked perfe...,1.0,1
179,gvgbq2j,-1,-1,-1,1,-1,"Yeah, I was going to sell my UWMC for a L, but...",-1.0,-1
180,h0g8s39,1,1,1,1,1,"Yep, it’s cheap right now and we could see it ...",1.0,1
181,h12l678,0,0,0,-1,-1,You didn’t know what a hedge fund was before G...,0.0,0


In [56]:
score = cohen_kappa_score(df["majority vote"], df["Own annotation"])

In [58]:
score

0.7203627939074284

In [62]:
fleiss_kappa(df[annotators].to_numpy())

AssertionError: 

In [67]:
from statsmodels.stats import inter_rater as irr
agg = irr.aggregate_raters(df[["majority vote", "Own annotation"]].to_numpy()) # returns a tuple (data, categories)
agg

(array([[0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 1, 1],
        [0, 2, 0],
        [0, 2, 0],
        [0, 2, 0],
        [0, 0, 2],
        [0, 0, 2],
        [0, 2, 0],
        [0, 0, 2],
        [0, 0, 2],
        [0, 2, 0],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 2, 0],
        [0, 1, 1],
        [0, 2, 0],
        [2, 0, 0],
        [1, 1, 0],
        [2, 0, 0],
        [0, 2, 0],
        [2, 0, 0],
        [0, 2, 0],
        [2, 0, 0],
        [0, 0, 2],
        [0, 2, 0],
        [0, 1, 1],
        [0, 2, 0],
        [1, 1, 0],
        [2, 0, 0],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2],
        [0, 0, 2],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 0],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 0, 2],
        [0, 1, 1],
        [0, 

In [68]:
irr.fleiss_kappa(agg[0], method='fleiss')

0.7198449344428257

## Scale evaluation

In [77]:
df["annotator avg"] = df[annotators].mean(axis=1)

In [79]:
df["MAE"] = df[["annotator avg", "Own annotation"]].apply(lambda x: abs(x[0] - x[1]), axis=1)

In [80]:
df

,id,ben,dliden,doss,jmo,jmo2,post,Own annotation,annotator avg,MAE
0,h15nzwn,2,2,2,2,2,"\n \nGentlemen, start your boners:\n\nBlackBe...",2.0,2.0,0.0
1,gyxowrt,2,2,2,2,2,#It has come to my attention that $PLTR Nation...,2.0,2.0,0.0
2,h0cufvd,2,2,2,2,2,#SNDL and TLRY for smooking weed on the moon 🚬...,2.0,2.0,0.0
3,h16r93i,2,2,2,2,2,$63k in on WKHS calls,2.0,2.0,0.0
4,h1adyxq,1,2,1,2,2,$CLF and $MT,1.0,1.6,0.6
...,...,...,...,...,...,...,...,...,...,...
178,gx0pu7v,2,2,2,2,2,Woke up on the floor got up and I looked perfe...,2.0,2.0,0.0
179,gvgbq2j,0,0,0,2,0,"Yeah, I was going to sell my UWMC for a L, but...",0.0,0.4,0.4
180,h0g8s39,2,2,2,2,2,"Yep, it’s cheap right now and we could see it ...",2.0,2.0,0.0
181,h12l678,1,1,1,0,0,You didn’t know what a hedge fund was before G...,1.0,0.6,0.4


In [81]:
MAE = df["MAE"].sum() / len(df["MAE"])

In [82]:
MAE

0.24043715846994532